In [ ]:
%pip install llama-index
%pip install llama-index-llms-ollama
%pip install llama-index-embeddings-huggingface
%pip install llama-index-core llama-index-readers-file 
%pip install -e llama-index-integrations/llms/llama-index-llms-ollama
%pip install llama-index-llms-replicate
%pip install llama-parse
%pip install replicate

In [ ]:
%pip install beir

In [ ]:
%pip install spacy

In [ ]:
%pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web

Making a cache directory to store cache files 


In [91]:
import os
os.environ['LLAMA_INDEX_CACHE_DIR'] = './h'

In [374]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama


def get_meta(file_path):
    return {"foo": "bar", "file_path": file_path}

reader = SimpleDirectoryReader("./data",recursive = True,file_metadata=get_meta)
#SimpleDirectoryReader(input_files=["path/to/file1", "path/to/file2"])
#SimpleDirectoryReader(
   # input_dir="path/to/directory", exclude=["path/to/file1", "path/to/file2"]
#)
#SimpleDirectoryReader(
   # input_dir="path/to/directory", required_exts=[".pdf", ".docx"]
#)
#SimpleDirectoryReader(input_dir="path/to/directory", num_files_limit=100)
all_docs = []
for docs in reader.iter_data():
    # <do something with the documents per file>
    all_docs.extend(docs)
"""
from llama_index.core import SimpleDirectoryReader
from llama_index.core.readers.base import BaseReader
from llama_index.core import Document


class MyFileReader(BaseReader):
    def load_data(self, file, extra_info=None):
        with open(file, "r") as f:
            text = f.read()
        # load_data returns a list of Document objects
        return [Document(text=text + "Foobar", extra_info=extra_info or {})]


reader = SimpleDirectoryReader(
    input_dir="./data", file_extractor={".myfile": MyFileReader()}
)

"""
documents = reader.load_data()
# Configure settings
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = Ollama(model="llama3", request_timeout=360.0, temperature=0.1)

# Create index
index = VectorStoreIndex.from_documents(documents)


In [ ]:
"""
from llama_index.core import VectorStoreIndex, download_loader

from llama_index.readers.google import GoogleDocsReader

gdoc_ids = ["1wf-y2pd9C878Oh-FmLH7Q_BQkljdm6TQal-c1pUfrec"]
loader = GoogleDocsReader()
documents = loader.load_data(document_ids=gdoc_ids)
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
query_engine.query("Where did the author go to school?")
"""

In [ ]:
documents

In [252]:
query_engine1 = index.as_query_engine(response_mode="compact")
response = query_engine1.query("What are the things in the Sprint planning checklist? and also list the page number")
print(response)

The Sprint planning checklist includes:

* Agenda
* Previous sprint summary
* Details
* Velocity tracking
* Adjusted velocity tracking
* Capacity planning
* Potential risks

Page number: Page 1


In [381]:
query_engine1 = index.as_query_engine(response_mode="refine")
response = query_engine1.query("Explain the error details and also list the page number")
print(response)

The issue at hand is an intermittent database connection timeout error affecting certain reports. This error impacts productivity during peak hours. The primary configuration item causing this problem is a database connection timeout. To resolve this issue, confirm that the error message received is related to a database connection timeout and verify that the network connectivity between the application and the database server is stable. Additionally, validate the database credentials (username, password, hostname, port) are correct.

Page number: 2


In [130]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)
from llama_index.core.indices import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

In [131]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [132]:

from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=True
)
query = "What are the things to be kept in mind before starting the meet?"
response = query_engine.query("What are the things to be kept in mind before starting the meet?")

print(response)

Before starting the meeting, it's essential to close the previous sprint, organize the backlog, and confirm team capacity updates. Additionally, reviewing how to estimate story points can help set expectations and goals for the upcoming sprint.


### Generating questions

In [165]:

from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
data_generator = DatasetGenerator.from_documents(documents)

/Users/chirag/Library/Python/3.9/lib/python/site-packages/llama_index/core/evaluation/dataset_generation.py:215: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(


In [166]:
eval_questions = data_generator.generate_questions_from_nodes()

/Users/chirag/Library/Python/3.9/lib/python/site-packages/llama_index/core/evaluation/dataset_generation.py:312: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [167]:
eval_questions

['Here are 10 questions based on the given context:',
 'What type of file is "template.pdf"?',
 'Who are the team members mentioned in the Sprint team members section?',
 'When was the last modified date of the file "template.pdf"?',
 'What is the purpose of the Sprint planning meeting items section?',
 'How many bytes does the file "template.pdf" weigh (based on its file size)?',
 'Who brought this template to you, according to the text?',
 'What is the name of the project and issue tracker that provided this template?',
 'When was the creation date of the file "template.pdf"?',
 'What are some of the topics covered in the Sprint planning resources section?',
 'What is the purpose of organizing the backlog, as mentioned in the Preparation Meeting Follow-up section?',
 'These questions cover various aspects of the given context, including file information, team members, and meeting preparation.',
 'Here are 10 questions based on the provided context information:',
 'What is the file ty

## Trial evaluation: didn't work

In [168]:
evaluator = RelevancyEvaluator(llm= Settings.llm)

In [180]:
from llama_index.core import Response
import pandas as pd

# Function to display evaluation results in a DataFrame
def display_eval_df(query: str, response: Response, eval_result) -> None:
    # Ensure eval_result is a list, even if it contains a single item
    if not isinstance(eval_result, list):
        eval_result = [eval_result]

    # Creating DataFrame
    eval_df = pd.DataFrame({
        "Query": [query] * len(eval_result),
        "Response": [str(response)] * len(eval_result),
        "Source": [response.source_nodes[0].node.get_content()[:1000] + "..."] * len(eval_result),
        "Evaluation Result": eval_result,
    })

    # Styling DataFrame
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "0px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

# Example usage
response_vector = query_engine.query(eval_questions[2])
eval_result = evaluator.evaluate_response(
    query=eval_questions[2], response=response_vector
)
display_eval_df(eval_questions[2], response_vector, eval_result)


,Query,Response,Source,Evaluation Result
0,Who are the team members mentioned in the Sprint team members section?,Ram SDE2 and Sita SDE1.,"JIRA Sprint planning checklist Sprint team members Sprint planning meeting items Agenda Previous sprint summary Details Velocity tracking Adjusted velocity tracking Capacity planning Potential risks Sprint planning resources Sprint boards and retrospectives Team resources and definitions Sprint planning checklist Sprint team members Sprint planning meeting items Use this template to structure your meeting, set expectations and goals, and define the backlog for the upcoming sprint. For detailed instructions and best practices, see our sprint planning guide and review how to estimate story points.This template is brought to you by Jira, Atlassian's project and issue tracker. Customize this template with information and resources your team uses each sprint to save prep time. Preparation Meeting Follow up organise the backlog and close the last sprintconfirm team capacity update Jira tickets Ram SDE2 Sita SDE1 Name Role...","query='Who are the team members mentioned in the Sprint team members section?' contexts=[""JIRA\n Sprint planning checklist\n Sprint team members\n Sprint planning meeting items\nAgenda\nPrevious sprint summary\nDetails\nVelocity tracking\nAdjusted velocity tracking\nCapacity planning\nPotential risks\n Sprint planning resources\nSprint boards and retrospectives\nTeam resources and definitions\n Sprint planning checklist\n \n Sprint team members\n \n Sprint planning meeting items\nUse this template to structure your meeting, set expectations and goals, and define the backlog for the upcoming sprint. For detailed \ninstructions and best practices, see our sprint planning guide and review how to estimate story points.This template is brought to you by Jira, Atlassian's project and issue tracker. Customize this template with information and resources \nyour team uses each sprint to save prep time.\nPreparation Meeting Follow up\norganise the backlog and \nclose the last sprintconfirm team capacity update Jira tickets\nRam SDE2\nSita SDE1\n Name Role"", 'Sprint planning resources\nSprint boards and retrospectives\n \nTeam resources and definitions'] response='Ram SDE2 and Sita SDE1.' passing=True feedback='YES' score=1.0 pairwise_source=None invalid_result=False invalid_reason=None"


### This is done so that in future: we can automate the ground truth part

In [186]:
from llama_index.core import Document
all_text = ""

for doc in documents:
    all_text += doc.text

giant_document = Document(text=all_text)

In [363]:
giant_document

Document(id_='86d67a13-8b3c-4dad-9076-16a55dacadf1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="JIRA\n Sprint planning checklist\n Sprint team members\n Sprint planning meeting items\nAgenda\nPrevious sprint summary\nDetails\nVelocity tracking\nAdjusted velocity tracking\nCapacity planning\nPotential risks\n Sprint planning resources\nSprint boards and retrospectives\nTeam resources and definitions\n Sprint planning checklist\n \n Sprint team members\n \n Sprint planning meeting items\nUse this template to structure your meeting, set expectations and goals, and define the backlog for the upcoming sprint. For detailed \ninstructions and best practices, see our sprint planning guide and review how to estimate story points.This template is brought to you by Jira, Atlassian's project and issue tracker. Customize this template with information and resources \nyour team uses each sprint to save prep time.\nPreparation 

## Generating questions

In [193]:
import os
import random
random.seed(42)

from llama_index.core import ServiceContext
from llama_index.core.prompts import Prompt
from llama_index.core.evaluation import DatasetGenerator

question_dataset = []
if os.path.exists("questions.txt"):
    with open("questions.txt", "r") as f:
        for line in f:
            question_dataset.append(line.strip())
else:
    # generate questions
    data_generator = DatasetGenerator.from_documents(
        [giant_document],
        text_question_template=Prompt(
            "A sample from the LlamaIndex documentation is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Using the documentation sample, carefully follow the instructions below:\n"
            "{query_str}"
        ),
        question_gen_query=(
            "You are an evaluator for a search pipeline. Your task is to write a single question "
            "using the provided documentation sample above to test the search pipeline. The question should "
            "reference specific names, functions, and terms. Restrict the question to the "
            "context information provided.\n"
            "Question: "
        ),
       
    )
    generated_questions = data_generator.generate_questions_from_nodes()

    # randomly pick 40 questions from each dataset
    generated_questions = random.sample(generated_questions, 40)
    question_dataset.extend(generated_questions)

    print(f"Generated {len(question_dataset)} questions.")

In [194]:
print(random.sample(question_dataset, 5))


['What is the risk mitigation strategy suggested if the team has a smaller buffer than expected?', 'What is the purpose of the Sprint planning meeting items section?', 'Who are the team members mentioned in the Sprint team members section?', 'Who are the two team members mentioned in the capacity planning section?', 'How many hours does Ram have available for the current sprint?']


## Taking two text files : questions.txt and ground_truth.txt
## We generate responses for all questions in questions.txt.
## We compare these responses with ground_truth.txt
## To find hallucination score of each
## and therefore average hallucination score = Accuracy

In [367]:
from sentence_transformers import CrossEncoder 
model = CrossEncoder("vectara/hallucination_evaluation_model")


In [221]:
def load_questions(file_path):
    with open(file_path, 'r') as file:
        questions = file.readlines()
    questions = [q.strip() for q in questions]  # Remove any leading/trailing whitespace
    return questions


In [282]:
questions = load_questions("questions.txt")

In [283]:
# Evaluate the model with the questions
def evaluate_model(questions):
    responses = []
    for question in questions:
        response = query_engine.query(question)
          # Assuming `model` is a callable that takes a question and returns a response
        responses.append(response)
    return responses

In [284]:
responses = evaluate_model(questions)

In [285]:

responses

[Response(response='Ram SDE2 and Sita SDE1.', source_nodes=[NodeWithScore(node=TextNode(id_='621c1c1c-0a32-4dc5-af03-c1954797b064', embedding=None, metadata={'page_label': '1', 'file_name': 'template.pdf', 'file_path': '/Users/chirag/Documents/data/data/template.pdf', 'file_type': 'application/pdf', 'file_size': 132191, 'creation_date': '2024-07-08', 'last_modified_date': '2024-07-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='39403c67-4dcc-442f-ae06-a80783ec6cad', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'template.pdf', 'file_path': '/Users/chirag/Documents/data/data/template.pdf', 'file_type': 'application/pdf', 'file_size': 132191, 'creation_date': '2024-07-0

In [286]:
responses[0].response

'Ram SDE2 and Sita SDE1.'

In [287]:
n = 0
res = []
while n<10:
    res.append(responses[n].response)
    n += 1


In [324]:
def load_ground_truth(file_path):
    with open(file_path, 'r') as file:
        truth = file.readlines()
        truth = [t.strip() for t in truth]  # Remove any leading/trailing whitespace'
        print(truth)
    return truth


In [326]:
gt = load_ground_truth("ground_truth.txt")

['Ram and Sita', 'To structure your meeting, set expectations and goals', 'Sprint boards and retrospectives and Team resources and definitions', 'Ram and Sita', 'Stabilizing the platform by addressing critical bugs', '8 points', '20 days', '70%', '40 hours', 'The team might need to load low priority work to backlog']


In [327]:
gt

['Ram and Sita',
 'To structure your meeting, set expectations and goals',
 'Sprint boards and retrospectives and Team resources and definitions',
 'Ram and Sita',
 'Stabilizing the platform by addressing critical bugs',
 '8 points',
 '20 days',
 '70%',
 '40 hours',
 'The team might need to load low priority work to backlog']

In [335]:
def evaluation(questions,res,n):
    sentence_pairs = []
    for question in questions:
        while n<10:
            sentence_pairs.append((questions[n], res[n]))
            n += 1
    return sentence_pairs

In [331]:
res

['Ram SDE2 and Sita SDE1.',
 'The purpose of the "Sprint planning meeting items" section is to structure the meeting, set expectations and goals, and define the backlog for the upcoming sprint.',
 'Some of the topics covered in the Sprint planning resources section include Sprint boards and retrospectives, Team resources and definitions.',
 'The two team members mentioned in the capacity planning section are Ram and Sita.',
 'Stabilizing the platform by addressing critical bugs.',
 'Story 2 has 8 points.',
 '20 days.',
 '70%',
 '30 hours.',
 'The team might need to load low priority work to backlog.']

In [369]:
sentence_pairs = evaluation(gt,res,0)

In [337]:
sentence_pairs

[('Ram and Sita', 'Ram SDE2 and Sita SDE1.'),
 ('To structure your meeting, set expectations and goals',
  'The purpose of the "Sprint planning meeting items" section is to structure the meeting, set expectations and goals, and define the backlog for the upcoming sprint.'),
 ('Sprint boards and retrospectives and Team resources and definitions',
  'Some of the topics covered in the Sprint planning resources section include Sprint boards and retrospectives, Team resources and definitions.'),
 ('Ram and Sita',
  'The two team members mentioned in the capacity planning section are Ram and Sita.'),
 ('Stabilizing the platform by addressing critical bugs',
  'Stabilizing the platform by addressing critical bugs.'),
 ('8 points', 'Story 2 has 8 points.'),
 ('20 days', '20 days.'),
 ('70%', '70%'),
 ('40 hours', '30 hours.'),
 ('The team might need to load low priority work to backlog',
  'The team might need to load low priority work to backlog.')]

In [370]:
score  = model.predict(sentence_pairs)
print(f"Hallucination score: {score}\n")

Hallucination score: [0.18242246 0.6483753  0.7640626  0.40664437 0.9990018  0.9985594
 0.92363805 0.9972184  0.0050105  0.9994702 ]



In [371]:
s = 0
n=10
for i in score:
    s = s + i
print("Accuracy of the 10 responses:",s/n * 100)

Accuracy of the 10 responses: 69.24402994569391


## A normal llama without RAG


In [83]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.llms.replicate import Replicate

llm_replicate = Replicate(model="meta/meta-llama-3-8b-instruct")

In [89]:
llm = Ollama(model="llama3", request_timeout=120.0)
response = llm.complete("What is in the sprint planning checklist?")

print(response)

A sprint planning checklist typically includes a list of items to review and confirm before starting a new sprint. Here are some common items that might be included:

1. **Sprint goals**: Review the overall objectives for the upcoming sprint, including any key results or deliverables.
2. **Sprint scope**: Confirm which features, user stories, or tasks will be included in the current sprint.
3. **Prioritization**: Verify the prioritization of items in the sprint backlog, ensuring that the most important and high-priority items are addressed first.
4. **Estimation**: Review the estimated effort required for each task or story, including any dependencies or assumptions made during estimation.
5. **Acceptance criteria**: Confirm what constitutes "done" for each item on the sprint backlog, including any acceptance tests or validation processes.
6. **Task assignments**: Verify who will be working on which tasks and ensure that everyone understands their responsibilities and deadlines.
7. **S

## Comparing between compact, refine and tree-summarise methods.


In [353]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("Summarise both the templates")

print(response)

The first template appears to be a general outline for sprint planning, including sections such as agenda, previous sprint summary, velocity tracking, capacity planning, potential risks, and resources. This template seems to provide a framework for teams to structure their meetings, set expectations and goals, and define the backlog for an upcoming sprint.

The second template is more specific to the preparation meeting follow-up, outlining tasks such as organizing the backlog, closing the last sprint, confirming team capacity updates, and updating Jira tickets. This template appears to be focused on the logistics of wrapping up one sprint and preparing for the next.


In [355]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("Summarise both the templates")

print(response)

The two templates are:

* A comprehensive framework for planning sprints, including setting goals, identifying tasks, and allocating responsibilities.
* A valuable resource that outlines key terms, roles, and expectations for team members, ensuring a unified understanding of their work.


In [357]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("Summarise both the templates")

print(response)

The first template appears to be a general outline for sprint planning, including sections such as agenda, previous sprint summary, velocity tracking, capacity planning, potential risks, and resources. This template seems to provide a framework for teams to structure their meetings and plan for upcoming sprints.

The second template is more specific to the preparation meeting follow-up, outlining tasks such as organizing the backlog, closing the last sprint, and updating team capacity in Jira tickets. It also lists the names and roles of two team members, Ram SDE2 and Sita SDE1. This template appears to be a checklist for teams to use after their sprint planning meetings to ensure all necessary steps are taken.


## Making an agent

In [358]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

In [361]:
response = agent.chat("What is the difference in the 2 templates?")
print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: summary
Action Input: {'input': 'Please provide the template differences'}
Observation: The template has three main sections: Agenda, Sprint Planning Meeting Items, and Sprint Team Members. The first section, Agenda, lists specific items to be discussed during the meeting, including closing the previous sprint and planning for the upcoming one.

The second section, Sprint Planning Meeting Items, provides a checklist of tasks to complete during the meeting, such as organizing the backlog, confirming team capacity, and reviewing story points.

The third section, Sprint Team Members, lists the names and roles of the team members involved in the sprint.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The two templates are different in that one is for a general meeting agenda, while the other is specifically designed for a Spr

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

# global
from llama_index.core import Settings

Settings.text_splitter = text_splitter

# per-index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter]
)
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter

pipeline = IngestionPipeline(transformations=[TokenTextSplitter(), ...])

nodes = pipeline.run(documents=documents)